In [ ]:
from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Lambda, Layer, Add, Multiply
from keras.models import Model, Sequential

import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale
import matplotlib.pyplot as plt

import argparse
import os

In [ ]:

Data_set= pd.read_csv("/home/effrancodelos/Deep_learnig/Data_Nova/COAD/COAD_COX/data_Merge_Colon_t.csv")
Data_set.head(n=20)


In [ ]:

Pacients = Data_set['Pacients']
#Radiation = Data_set['radiations']


In [ ]:
Data_set.drop(['Pacients'], axis=1, inplace=True)
#Data_set.drop(Data_set.columns[0], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler    = MinMaxScaler()
df_norm   = scaler.fit_transform(Data_set)


In [ ]:
df_norm

In [ ]:
original_dim= df_norm.shape[1]
input_shape = (original_dim, )
intermediate_dim = int(original_dim/2)
batch_size = 128
latent_dim = 100
epochs     = 80
epsilon_std = 1.0

In [ ]:
class KLDivergenceLayer(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_var = inputs

        kl_batch = - .5 * K.sum(1 + log_var -
                                K.square(mu) -
                                K.exp(log_var), axis=-1)

        self.add_loss(K.mean(kl_batch), inputs=inputs)

        return inputs

In [ ]:
# VAE Architecture
# * original_dim - Original Input Dimension
# * intermediate_dim - Hidden Layer Dimension
# * latent_dim - Latent/Embedding Dimension
def vae_arc(original_dim, intermediate_dim, latent_dim):
    # Decode
    decoder = Sequential([
        Dense(intermediate_dim, input_dim=latent_dim, activation='relu'),
        Dense(original_dim, activation='sigmoid')
    ])

    # Encode
    x = Input(shape=(original_dim,))
    h = Dense(intermediate_dim, activation='relu')(x)

    z_mu = Dense(latent_dim)(h)
    z_log_var = Dense(latent_dim)(h)

    z_mu, z_log_var = KLDivergenceLayer()([z_mu, z_log_var])
    z_sigma = Lambda(lambda t: K.exp(.5*t))(z_log_var)

    eps = Input(tensor=K.random_normal(stddev=epsilon_std,
                                       shape=(K.shape(x)[0], latent_dim)))
    z_eps = Multiply()([z_sigma, eps])
    z = Add()([z_mu, z_eps])

    x_pred = decoder(z)
    
    return x, eps, z_mu, x_pred

In [ ]:
def nll(y_true, y_pred):
    """ Negative log likelihood (Bernoulli). """

    # keras.losses.binary_crossentropy gives the mean
    # over the last axis. we require the sum
    return K.sum(K.binary_crossentropy(y_true, y_pred), axis=-1)

In [ ]:
#Netwrok Configuration
x, eps, z_mu, x_pred = vae_arc(original_dim, intermediate_dim, latent_dim)
vae            = Model(inputs=[x, eps], outputs=x_pred)
vae.compile(optimizer='adam', loss=nll)

In [ ]:
vae.summary()


In [ ]:
#Data split
from sklearn.model_selection import train_test_split

# 
X_train, X_test, y_train, y_test = train_test_split(df_norm, df_norm, 
                                                    test_size=0.33, random_state=42)

In [ ]:
filepath   ="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
hist = vae.fit(X_train, X_train,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks_list,
        validation_data=(X_test, X_test))

In [ ]:
def plt_hist(hist):
    # summarize history for loss
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')

In [ ]:
plt_hist(hist)

In [ ]:
encoder = Model(x, z_mu)
z_df    = encoder.predict(df_norm, batch_size=batch_size)

In [ ]:
encoded_train = pd.DataFrame(encoder.predict(df_norm, batch_size=batch_size))
encoded_train = encoded_train.add_prefix('feature_')

In [ ]:
encoded_train['Pacients'] = Pacients

In [ ]:
encoded_train

In [ ]:
encoded_train.to_csv("/home/effrancodelos/Deep_learnig/Data_Nova/COAD/COAD_COX/data_Merge_Colon_COX_Variotional_1.csv", index=False)